In [27]:
# !conda install -c conda-forge python-docx 

In [68]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup 
from tqdm import tqdm

import docx

### Загрузка данных

Просмотр файлов ._json_.

Стратегия: Соединяем все файлы в один (создаем список файлов, добавляем по одному в общий файл и затем обрабатываем все вместе).
Стратегия 2.0: Просматриваем отдельно все файлы в папке и добавляем данные в датасет.

In [584]:
import glob
import os
import json
from collections import defaultdict

In [590]:
# path = os.path.abspath("Data")
# print(glob.glob(path+"/*.json"))

# all_filenames = []

# for i in glob.glob(path+"/*.json"):
#     all_filenames.append(i.split('/')[-1].split('.')[0])

# print(all_filenames)

['Example', 'MMM']


{

    "info": null, 
    
    "refs": [["text", "12.02.2002", 123]], 
    
    "desc": {
    
        "month": "12", 
        
        "rate": 1
        
     }
     
}

In [593]:
data_for_df = defaultdict(list)

for filename in glob.glob('./Data/*.json'): # все файлы формата json в папке
    with open(filename, 'r', encoding='utf-8') as f:
        data = json.load(f)
        for ref in data['refs']:
            if ref is not None:
                data_for_df['text'].append(ref[0])
                data_for_df['date'].append(ref[1])
                data_for_df['views'].append(ref[2])
                data_for_df['company_desc'].append(data['desc']['month'])
                data_for_df['company_rate'].append(data['desc']['rate'])
                data_for_df['company_name'].append(filename.split('/')[-1].split('.')[0])

In [596]:
data_for_df = pd.DataFrame(data_for_df)
data_for_df

,text,date,views,company_desc,company_rate,company_name
0,text,12.02.2002,123,12,1,Example
1,text,12.02.2002,123,12,1,MMM


### Парсинг

Сайта Habr (habr.com/ru/search/)

In [82]:
filename = 'Condidates.docx'
document = docx.Document(filename)

companies = [] # массив с именами компаний
for p in document.paragraphs:
    companies.append(p.text)

**Ф-ия поиска компании в основном поисковике**

In [602]:
companies_info = pd.DataFrame(np.zeros((1, 6)))
companies_info.columns = ['company', 'motto', 'rate', 'subs', 'description', 'access']

In [603]:
def get_company_desc(access):
    url = 'https://habr.com' + access
    r = requests.get(url)
    if r.status_code == 200:
        soup = BeautifulSoup(r.text, 'html5lib')
        description = soup.find('span', class_='tm-company-profile__content').find('span').text
        return description
    else:
        return None

def get_company_info(comp):
    global companies_info
    url = []
    link = 'https://habr.com/ru/search/?q='+comp+'&target_type=companies&order=relevance' # поиск во вкладке 'Компания'
    r = requests.get(link) 
    if r.status_code == 200:
        soup = BeautifulSoup(r.text, 'html5lib')
        doc = {}
        try:
            doc['company'] = comp
            doc['motto'] = soup.find('div', class_='tm-company-snippet__description').text
            doc['rate'] = soup.find('span', class_='tm-search-companies__score-counter tm-search-companies__score-counter_rating').text.split('\n')[1]
            doc['subs'] = soup.find('span', class_='tm-search-companies__score-counter tm-search-companies__score-counter_subscribers').text.split('\n')[1]   
            access = soup.find('div', class_='tm-company-snippet').find('a', class_='tm-company-snippet__logo-link').get('href')
            doc['description'] = get_company_desc(access)
            doc['access'] = access
            companies_info = companies_info.append(doc, ignore_index=True) # загружает в отдельную выборку о компаниях
        except Exception as exc:
            print(exc)
    else:
        return None

for company in companies:
    get_company_info(company)

'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute

In [604]:
companies_info = companies_info.drop(0)
companies_info

,company,motto,rate,subs,description,access
1,X5 Group,Все о цифровой трансформации ритейла,81.1,7.4K,X5 Group — это одна из ведущих компаний рынка ...,/ru/company/X5Group/profile/
2,Нетология,Образовательная платформа,99.45,47K,Нетология обучает современным востребованным п...,/ru/company/netologyru/profile/
3,Иннотех,We help the businesses with digital transforma...,25.68,87,Мы мыслим инновациями\n\nГК «Иннотех» — это со...,/ru/company/innotech/profile/
4,Skillbox,Образовательная платформа Skillbox,100.32,71K,Образовательная платформа Skillbox — один из л...,/ru/company/skillbox/profile/
5,Билайн,Компания,115.1,7.2K,ОАО «ВымпелКом» входит в группу компаний «Вымп...,/ru/company/beeline/profile/
6,Московская биржа,Инвестиции начинаются здесь,82.64,535,Московская биржа (входит в MOEX Group) – это к...,/ru/company/moex/profile/


Всего 6 компаний с заполненной информацией внутри.

In [605]:
all_links = []
all_comp_names = [] # создадим библиотеку с названиями компаний в поисковой строке
for comp in companies_info.company:
    access = companies_info[companies_info.company == comp].access
    access = access.reset_index(drop=True) 
    access = str(access)
    company_name = access.split('/')[3]
    link = 'https://habr.com/ru/company/' + company_name + '/blog/'
    print(link)
    all_comp_names.append(company_name)
    all_links.append(link)

https://habr.com/ru/company/X5Group/blog/
https://habr.com/ru/company/netologyru/blog/
https://habr.com/ru/company/innotech/blog/
https://habr.com/ru/company/skillbox/blog/
https://habr.com/ru/company/beeline/blog/
https://habr.com/ru/company/moex/blog/


**Ф-ия нахождения id постов**

In [606]:
def get_page_data(comp):
    url = []
    link = 'https://habr.com/ru/company/' + all_comp_names[build_access(comp)] + '/blog/'
    r = requests.get(link)
    if r.status_code == 200:
        soup = BeautifulSoup(r.text, 'html5lib')
    else:
        return None
    dir = soup.find('div', class_='tm-articles-list')
    news_url = dir.find_all('article', class_='tm-articles-list__item')
    for add in news_url:
        try:
            url.append(add.get('id'))
        except:
            continue
    return url

In [607]:
# get_page_data('Иннотех')

['658007',
 '655505',
 '653821',
 '653501',
 '651781',
 '648415',
 '594579',
 '592643',
 '591623',
 '585332',
 '583406',
 '579044',
 '576512',
 '570472',
 '569942',
 '565504',
 '554018',
 '552360',
 '541352',
 '540940']

**Ф-ия поиска постов компании**

In [749]:
df = pd.DataFrame(np.zeros((1, 6)))
df.columns = ['company', 'description', 'rate', 'blog_post', 'tags', 'date']

In [750]:
def get_blog_content(url, comp):
    global df
    r = requests.get(url)
    if r.status_code == 200:
        soup = BeautifulSoup(r.text, 'html5lib')
        doc = {}
        try:
            doc['company'] = comp
            doc['description'] = companies_info[companies_info['company'] == comp].description
            doc['rate'] = soup.find('div', class_='tm-rating__counter').text
            dir = soup.find('div', class_='article-formatted-body')
            doc['blog_post'] = dir.find('div').text
            dir = soup.find('ul', class_='tm-separated-list__list')
            tags = dir.find_all('li', class_='tm-separated-list__item')
            all_tags = []
            for tag in tags:
                tag_text = tag.find('a', class_='tm-tags-list__link').text
                all_tags.append(tag_text)
            doc['tags'] = all_tags
    #             hubs = soup.find_all('span', class_='tm-article-snippet__hubs-item')
    #             all_hubss = []
    #             for hub in hubs:
    #                 hub_text = hub.find('span').text
    #                 all_hubss.append(hub_text)
    #             doc['hubs'] = all_hubss
            doc['date'] = soup.find('span', class_='tm-article-snippet__datetime-published').text
            df = df.append(doc, ignore_index=True)
        except Exception as exc:
            print(exc, 'it doesnt work')
    else:
        return None

In [742]:
# get_blog_content('https://habr.com/ru/company/X5Group/blog/597779/', 'X5 Group')

In [743]:
# df

In [744]:
# def get_last_page(comp):
#     link = 'https://habr.com/ru/search/?q=' + comp + '&target_type=posts&order=relevance'
#     r = requests.get(link)
#     if r.status_code == 200:
#         soup = BeautifulSoup(r.text, 'html5lib')
#         print('works')
#     else:
#         return None
#     pages_url = soup.find('div', class_='tm-articles-list').find_all('article', class_='tm-articles-list__item')
#     last_page = int(soup.find_all('div', class_='tm-pagination__page-group')[1].find_all('a', class_='tm-pagination__page')[0].text)
#     return last_page

# def get_article_content(url, comp):
#     global df
#     r = requests.get(url)
#     if r.status_code == 200:
#         soup = BeautifulSoup(r.text, 'html5lib')
#         doc = {}
#         try:
#             doc['title'] = soup.find('h1', class_='tm-article-snippet__title_h1').text
#             dir = soup.find('div', class_='article-formatted-body')
#             doc['description'] = dir.find('div').text
#             doc['rate'] = soup.find('div', class_='tm-rating__counter').text
#             doc['company'] = comp
#             dir = soup.find('ul', class_='tm-separated-list__list')
#             tags = dir.find_all('li', class_='tm-separated-list__item')
#             all_tags = []
#             for tag in tags:
#                 tag_text = tag.find('a', class_='tm-tags-list__link').text
#                 all_tags.append(tag_text)
#             doc['keywords'] = all_tags
#             hubs = soup.find_all('span', class_='tm-article-snippet__hubs-item')
#             all_hubss = []
#             for hub in hubs:
#                 hub_text = hub.find('span').text
#                 all_hubss.append(hub_text)
#             doc['hubs'] = all_hubss
#             doc['date'] = soup.find('span', class_='tm-article-snippet__datetime-published').text
#             df = df.append(doc, ignore_index=True)
#         except Exception as exc:
#             print(exc)
#     else:
#         return None

**Ф-ия парсера**

In [745]:
import warnings
warnings.filterwarnings("ignore")

In [746]:
all_companies_in_habr = companies_info.company.unique()
all_companies_in_habr 

array(['X5 Group', 'Нетология ', 'Иннотех', 'Skillbox', 'Билайн',
       'Московская биржа'], dtype=object)

In [751]:
def build_access(comp):
#     access = companies_info[companies_info.company == comp].index
#     access = str(access)
#     access = access.split('(')[1].split(']')[0].split('[')[1]
#     access = int(access)
#     return access-1
    if comp == 'X5 Group':
        access = 0
    elif comp == 'Нетология':
        access = 1
    elif comp == 'Иннотех':
        access = 2
    elif comp == 'Skillbox':
        access = 3
    elif comp == 'Билайн':
        access = 4
    else:
        access = 5
    return access

build_access('Нетология')

# access = companies_info[companies_info.company == 'Нетология'].index
# # access = str(access)
# access = access #.split('(')[1].split(']')[0].split('[')[0]
# # access = int(access)
# access

1

In [752]:
companies_info.company

1            X5 Group
2          Нетология 
3             Иннотех
4            Skillbox
5              Билайн
6    Московская биржа
Name: company, dtype: object

In [753]:
def parser():
    count = 0
    for company in all_companies_in_habr:
        urls = get_page_data(company)
        for url in urls:
            link = 'https://habr.com/ru/company/' + all_companies_in_habr[build_access(comp)] + '/blog/' + url + '/'
            get_blog_content(link, company)
            count += 1
            print(count, link)

In [754]:
tqdm(parser())

1 https://habr.com/ru/company/Московская биржа/blog/654115/
2 https://habr.com/ru/company/Московская биржа/blog/653961/
3 https://habr.com/ru/company/Московская биржа/blog/653571/
4 https://habr.com/ru/company/Московская биржа/blog/652387/
5 https://habr.com/ru/company/Московская биржа/blog/651739/
6 https://habr.com/ru/company/Московская биржа/blog/651451/
7 https://habr.com/ru/company/Московская биржа/blog/651057/
8 https://habr.com/ru/company/Московская биржа/blog/650651/
9 https://habr.com/ru/company/Московская биржа/blog/647365/
10 https://habr.com/ru/company/Московская биржа/blog/598919/
11 https://habr.com/ru/company/Московская биржа/blog/645651/
12 https://habr.com/ru/company/Московская биржа/blog/597779/
'NoneType' object has no attribute 'text' it doesnt work
13 https://habr.com/ru/company/Московская биржа/blog/597491/
14 https://habr.com/ru/company/Московская биржа/blog/596279/
15 https://habr.com/ru/company/Московская биржа/blog/595387/
16 https://habr.com/ru/company/Москов





0it [00:00, ?it/s]

In [761]:
df.head(5)

,company,description,rate,blog_post,tags,date
0,0.0,0.0,0.0,0.0,0.0,0.0
1,X5 Group,1 X5 Group — это одна из ведущих компаний р...,-0.6,Привет! Недавно X5 провела митап об опенсорсно...,"[x5retailgroup, keycloak, базы данных, открыты...",3 марта в 16:21
2,X5 Group,1 X5 Group — это одна из ведущих компаний р...,-0.6,"Привет! Меня зовут Сергей Тетерюков, и я работ...","[x5retailgroup, база данных, big data, cassand...",1 марта в 16:27
3,X5 Group,1 X5 Group — это одна из ведущих компаний р...,-0.6,X5 Tech приглашает студентов и выпускников вуз...,"[x5retailgroup, обучение, big data, большие да...",26 февраля в 15:38
4,X5 Group,1 X5 Group — это одна из ведущих компаний р...,0,"Привет! Меня зовут Кирилл Цветков. Я DevOps, и...","[devops, разработка, администрирование, спасиб...",18 февраля в 18:00


In [776]:
df = df.drop(0)

In [778]:
df.blog_post.unique().shape

(81,)

In [781]:
# Добавляем целевую выборку для будущей кластеризации
dict_target = {
    'Благотворительный фонд “Подари жизнь”': 'Государство и Общество',
    'АНО “Диалог”': 'Государство и Общество',
    'Правительство Москвы, проект Цифровая экосистема Москвы': 'Государство и Общество',
    'ФНС России': 'Государство и Общество',
    'Пушкинская карта': 'Государство и Общество',
    'Телеканал 360': 'СМИ и Массовые коммуникации',
    'Медиахолдинг Independent Media': 'СМИ и Массовые коммуникации',
    'X5 Group': 'СМИ и Массовые коммуникации',
    'Коммерсантъ': 'СМИ и Массовые коммуникации',
    'Электронный дневник (МЭШ)': 'Образование и Кадры',
    'Проект программы “Сбер для детей и родителей” совместно с Rambler&Co': 'Образование и Кадры',
    'ООО “Цифровое образование”, проект ИКОП “Сферум”': 'Образование и Кадры',
    'Нетология ': 'Образование и Кадры',
    'Компания Samsung Electronics': 'Образование и Кадры',
    'Gett': 'Наука, технологии и инновации',
    'Иннотех': 'Наука, технологии и инновации',
    'Skillbox': 'Наука, технологии и инновации',
    'Промобот': 'Наука, технологии и инновации',
    'SberDevices и Rambler&Co ': 'Наука, технологии и инновации',
    'Билайн': 'Наука, технологии и инновации',
    'Общественное телевидение России': 'Культурные проекты в Сети',
    'Проект “Путь героя” информационного агентства ТАСС и киностудии “Союзмультфильм”': 'Культурные проекты в Сети',
    'Эмпатия Манучи (авторский канал Манучарова Вячеслава)': 'Культурные проекты в Сети',
    'Федеральное государственное бюджетное учреждение культуры "Всероссийское музейное объединение “Государственная Третьяковская галерея”': 'Культурные проекты в Сети',
    'Федерация компьютерного спорта города Москвы — “Московский Киберспорт”\xa0': 'Игровая индустрия и киберспорт',
    'ESforce Holding': 'Игровая индустрия и киберспорт',
    'Cybersport.Metaratings': 'Игровая индустрия и киберспорт',
    'MetaMMA': 'Игровая индустрия и киберспорт',
    'Goose Gaming': 'Игровая индустрия и киберспорт',
    'Федерация креативных индустрий': 'Креативные индустрии',
    'Гэллэри Сервис': 'Креативные индустрии',
    'Vinci Agency': 'Креативные индустрии',
    'Студия Артемия Лебедева': 'Креативные индустрии',
    'Здоровье.ру': 'Здоровье и медицина',
    'Здоровое Питание': 'Здоровье и медицина',
    'Проект по использованию технологий компьютерного зрения на базе искусственного интеллекта (ИИ) для анализа медицинских изображений': 'Здоровье и медицина',
    'СберМаркет': 'Экономика и бизнес',
    'Balance Platform': 'Экономика и бизнес',
    'Московская биржа': 'Экономика и бизнес',
    'Метарейтинг': 'Экономика и бизнес',
    'Портал для предпринимателей Smallbusiness.ru': 'За развитие среднего и малого бизнеса в Рунете',
    'ПАО "МТС", проект “Знаем лично”': 'За развитие среднего и малого бизнеса в Рунете',
    'Банк ВТБ ПАО, проект интернет-банк': 'За развитие среднего и малого бизнеса в Рунете',
    'Автономная некоммерческая организация по развитию экологических, социальных и патриотических проектов “Сад Памяти”': '#СЧАСТЬЕЕСТЬ',
    'Oh My Град': 'Развитие внутреннего туризма',
    'Россия: 85 приключений, проект онлайн-медиа Russia Beyond, входящее в телесеть RT': 'Развитие внутреннего туризма',
    'АНО “Проектный офис по развитию туризма и гостеприимства Москвы” Туристический сервис RUSSPASS': 'Развитие внутреннего туризма',
    'Discovermoscow.com': 'Развитие внутреннего туризма',
    'Мэр Вологды — Сергей Воропанов': 'Открытые коммуникации',
    'Правительство Тульской области': 'Открытые коммуникации',
    'Russia.travel': 'Открытые коммуникации'
}

In [782]:
all_target = []

for company in df.company:
    all_target.append(dict_target[company])
df['target'] = all_target

In [783]:
df.to_csv('dataset_parsed.csv', index=False)

In [ ]:
# ОТФОРМАТИРОВАТЬ DF
df = df

In [ ]:
# СОЕДИНИТЬ ДАТА JSON И DF
df = df.append(data, ignore_index=True)

### Предобработка

In [786]:
import re
import pymorphy2
from functools import lru_cache

morph = pymorphy2.MorphAnalyzer()

**Импорт стоп-слов**

(Ссылка на гитхаб)[https://github.com/stopwords-iso/stopwords-ru/blob/master/stopwords-ru.txt]

In [785]:
stop_words = list(map(lambda x: x.strip(), open('stop_words.txt').readlines()))

**Лемматизация**

Ф-ия приведения слов к их начальной форме.

In [787]:
@lru_cache() # динамика
def word2norm(word):
    return morph.parse(word)[0].normal_form

**Ф-ия предобработки**

Построения языкового корпуса.

In [800]:
word2normal_form = {}

def preprocess_text(text):
#     if pd.isna(text):
#         return ''
    
    clean_text = re.sub(re.compile('<.*?>'), '', str(text)) # очистка данных - удаление тегов html
    clean_text = re.sub('[^А-Яа-яA-Za-z]', ' ', str(text)) # очистка данных - удаление всех символов, не относящихся к буквам
    clean_text = clean_text.lower() # изменение на нижний регистр
    
    norm_text_words = []
    for word in clean_text.split():
        if word in stop_words:
            continue # проверка на наличие слова в списке стоп-слов
        if word not in word2normal_form:
            normal_form = morph.parse(word)[0].normal_form # лемматизация с динамикой
            word2normal_form[word] = normal_form
        else:
            normal_form = word2normal_form[word]
        norm_text_words.append(normal_form)
        
    norm_text = ' '.join(norm_text_words)
    return norm_text

In [802]:
pd.isna(df.description).any()

False

In [809]:
clean_descriptions = []
clean_posts = []

for text in tqdm(df.description):
    clean_desc = preprocess_text(text)
    clean_descriptions.append(clean_desc)
    
for text in tqdm(df.blog_post):
    clean_post = preprocess_text(text)
    clean_posts.append(clean_post)






  0%|                                                                      | 0/101 [00:00<?, ?it/s]




100%|███████████████████████████████████████████████████████████| 101/101 [00:00<00:00, 897.95it/s]





  0%|                                                                      | 0/101 [00:00<?, ?it/s]




  5%|███                                                           | 5/101 [00:00<00:01, 49.86it/s]




 12%|███████▏                                                     | 12/101 [00:00<00:01, 60.49it/s]




 21%|████████████▋                                                | 21/101 [00:00<00:01, 72.52it/s]




 29%|█████████████████▌                                           | 29/101 [00:00<00:01, 68.03it/s]




 41%|████████████████████████▊                                    | 41/101 [00:00<00:00, 85.20it/s]




 50%|██████████████████████████████▏                              | 50/101 [00:00<00:00, 81.03it/s]




 58%|███████████████████████████████████▋                 

In [811]:
df['clean_posts'] = clean_posts
df['clean_descriptions'] = clean_descriptions

In [821]:
df.head(3)

,company,description,rate,blog_post,tags,date,target,clean_posts,clean_descriptions
1,X5 Group,1 X5 Group — это одна из ведущих компаний р...,-0.6,Привет! Недавно X5 провела митап об опенсорсно...,"[x5retailgroup, keycloak, базы данных, открыты...",3 марта в 16:21,СМИ и Массовые коммуникации,привет x провести митапа опенсорсный проект ke...,x group один ведущий компания рынок name descr...
2,X5 Group,1 X5 Group — это одна из ведущих компаний р...,-0.6,"Привет! Меня зовут Сергей Тетерюков, и я работ...","[x5retailgroup, база данных, big data, cassand...",1 марта в 16:27,СМИ и Массовые коммуникации,привет звать сергей тетерюков работать инженер...,x group один ведущий компания рынок name descr...
3,X5 Group,1 X5 Group — это одна из ведущих компаний р...,-0.6,X5 Tech приглашает студентов и выпускников вуз...,"[x5retailgroup, обучение, big data, большие да...",26 февраля в 15:38,СМИ и Массовые коммуникации,x tech приглашать студент выпускник вуз пройти...,x group один ведущий компания рынок name descr...


In [822]:
df.to_csv('dataset_cleaned.csv', index=False)